In [2]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
#from sklearn import linear_model as lm
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
dataset = pd.read_csv("crime.csv")

# Classical statistics approach
### Forward selection

In [4]:
null_lm = ols('murder_rate ~ 1', data=dataset).fit()

# dictionary of simple regressions
single_var_models = {
    "poverty": ols('murder_rate ~ poverty', data=dataset).fit(),
    "high_school": ols('murder_rate ~ high_school', data=dataset).fit(),
    "college": ols('murder_rate ~ college', data=dataset).fit(),
    "single_parent": ols('murder_rate ~ single_parent', data=dataset).fit(),
    "unemployed": ols('murder_rate ~ unemployed', data=dataset).fit(),
    "metropolitan": ols('murder_rate ~ metropolitan', data=dataset).fit(),
    "region": ols('murder_rate ~ region', data=dataset).fit() # baseline: north-central
}


In [26]:
for key in single_var_models:
    results = sm.stats.anova_lm(single_var_models[key], typ=3)
    print(results)
    # R^2 = (intercept + explanatory variable(s) SS) / total SS
    # (total-error)/total -> 1 - error/total
    r_squared = 1 - results["sum_sq"]["Residual"]/results["sum_sq"].sum()
    print("R^2 =", round(r_squared,3), "\n")

               sum_sq    df          F    PR(>F)
Intercept    0.565516   1.0   0.107489  0.744447
poverty     56.224089   1.0  10.686658  0.001999
Residual   252.535111  48.0        NaN       NaN
R^2 = 0.184 

                 sum_sq    df          F        PR(>F)
Intercept    145.505447   1.0  35.493964  2.909426e-07
high_school  111.985984   1.0  27.317372  3.710570e-06
Residual     196.773216  48.0        NaN           NaN
R^2 = 0.567 

               sum_sq    df          F    PR(>F)
Intercept   92.095520   1.0  15.168256  0.000304
college     17.322599   1.0   2.853055  0.097687
Residual   291.436601  48.0        NaN       NaN
R^2 = 0.273 

                   sum_sq    df          F        PR(>F)
Intercept       57.354449   1.0  16.474017  1.808947e-04
single_parent  141.646736   1.0  40.685435  6.605019e-08
Residual       167.112464  48.0        NaN           NaN
R^2 = 0.544 

                sum_sq    df         F    PR(>F)
Intercept     2.575526   1.0  0.456709  0.502409
unempl

The highest $R^2$ value is 0.387, belonging to the `region` variable, so we'll start with that in the model (I'm going to assume, for this data, that this is better than the null model).

In [27]:
present_model = single_var_models["region"]

Now let's add another variable and see how these two-varable models compare to our present model. I'll look for the largest $R^2$ value of the two-variable models, and see if that is an improvement over the single-variable one using an $F$-test.

In [28]:
double_var_models = {
    "poverty": ols('murder_rate ~ region + poverty', data=dataset).fit(),
    "high_school": ols('murder_rate ~ region + high_school', data=dataset).fit(),
    "college": ols('murder_rate ~ region + college', data=dataset).fit(),
    "single_parent": ols('murder_rate ~ region + single_parent', data=dataset).fit(),
    "unemployed": ols('murder_rate ~ region + unemployed', data=dataset).fit(),
    "metropolitan": ols('murder_rate ~ region + metropolitan', data=dataset).fit(),
}

In [30]:
for key in double_var_models:
    results = sm.stats.anova_lm(double_var_models[key], typ=3)
    print(results)
    # R^2 = (intercept + explanatory variable(s) SS) / total SS
    # (total-error)/total -> 1 - error/total
    r_squared = 1 - results["sum_sq"]["Residual"]/results["sum_sq"].sum()
    print("R^2 =", round(r_squared,3), "\n")

               sum_sq    df         F    PR(>F)
Intercept    8.422499   1.0  1.904812  0.174359
region      53.558856   3.0  4.037581  0.012611
poverty     17.095555   1.0  3.866290  0.055452
Residual   198.976256  45.0       NaN       NaN
R^2 = 0.284 

                 sum_sq    df          F    PR(>F)
Intercept     54.596671   1.0  14.065382  0.000502
region        22.099668   3.0   1.897797  0.143538
high_school   41.398263   1.0  10.665163  0.002092
Residual     174.673548  45.0        NaN       NaN
R^2 = 0.403 

               sum_sq    df         F    PR(>F)
Intercept   15.838482   1.0  3.299052  0.075985
region      75.395240   3.0  5.234778  0.003478
college      0.030450   1.0  0.006343  0.936876
Residual   216.041361  45.0       NaN       NaN
R^2 = 0.297 

                   sum_sq    df          F    PR(>F)
Intercept       31.708943   1.0  10.735430  0.002029
region          34.197206   3.0   3.859287  0.015354
single_parent   83.156553   1.0  28.153614  0.000003
Residual   

The model with `single_parent` has the best $R^2$ of all of these candidates, so we'll see if that is significantly better than the current model with an $F$-test.

In [32]:
print(sm.stats.anova_lm(present_model, double_var_models["single_parent"], test="F"))

   df_resid         ssr  df_diff    ss_diff          F    Pr(>F)
0      46.0  216.071811      0.0        NaN        NaN       NaN
1      45.0  132.915258      1.0  83.156553  28.153614  0.000003


It's definitely significant! Let's update our current model.

In [33]:
present_model = double_var_models["single_parent"]

Now let's make some 3-variable models and compare them to our current model.

In [10]:
triple_var_models = {
    "poverty": ols('murder_rate ~ region + single_parent + poverty', data=dataset).fit(),
    "high_school": ols('murder_rate ~ region + single_parent + high_school', data=dataset).fit(),
    "college": ols('murder_rate ~ region + single_parent + college', data=dataset).fit(),
    "unemployed": ols('murder_rate ~ region + single_parent + unemployed', data=dataset).fit(),
    "metropolitan": ols('murder_rate ~ region + single_parent + metropolitan', data=dataset).fit(),
}

In [34]:
for key in triple_var_models:
    results = sm.stats.anova_lm(triple_var_models[key], typ=3)
    print(results)
    # R^2 = (intercept + explanatory variable(s) SS) / total SS
    # (total-error)/total -> 1 - error/total
    r_squared = 1 - results["sum_sq"]["Residual"]/results["sum_sq"].sum()
    print("R^2 =", round(r_squared,3), "\n")

                   sum_sq    df          F    PR(>F)
Intercept       32.034172   1.0  10.636520  0.002145
region          29.023053   3.0   3.212239  0.031921
single_parent   66.460777   1.0  22.067416  0.000026
poverty          0.399779   1.0   0.132741  0.717352
Residual       132.515479  44.0        NaN       NaN
R^2 = 0.491 

                   sum_sq    df          F    PR(>F)
Intercept        3.273886   1.0   1.179137  0.283445
region          25.521452   3.0   3.063972  0.037742
single_parent   52.507083   1.0  18.911177  0.000080
high_school     10.748793   1.0   3.871332  0.055441
Residual       122.166465  44.0        NaN       NaN
R^2 = 0.43 

                   sum_sq    df          F    PR(>F)
Intercept       30.306274   1.0  10.377223  0.002402
region          38.114550   3.0   4.350289  0.009074
single_parent   87.541079   1.0  29.975090  0.000002
college          4.414976   1.0   1.511739  0.225408
Residual       128.500281  44.0        NaN       NaN
R^2 = 0.555 

     

`metropolitan` is far and away the variable that helps the model perform better, so let's see what the $F$-test says.

In [35]:
print(sm.stats.anova_lm(present_model, triple_var_models["metropolitan"], test="F"))

   df_resid         ssr  df_diff    ss_diff         F    Pr(>F)
0      45.0  132.915258      0.0        NaN       NaN       NaN
1      44.0  107.387079      1.0  25.528179  10.45973  0.002317


Definitely better! Let's update the model.

In [36]:
present_model = triple_var_models["metropolitan"]

Onto a fourth variable and beyond with the same process. I'll keep going until the $F$-test result is insignificant (for this case, $p < 0.05$).

In [37]:
four_var_models = {
    "poverty": ols('murder_rate ~ region + single_parent + metropolitan + poverty', data=dataset).fit(),
    "high_school": ols('murder_rate ~ region + single_parent + metropolitan + high_school', data=dataset).fit(),
    "college": ols('murder_rate ~ region + single_parent + metropolitan + college', data=dataset).fit(),
    "unemployed": ols('murder_rate ~ region + single_parent + metropolitan + unemployed', data=dataset).fit(),
}

In [38]:
for key in four_var_models:
    results = sm.stats.anova_lm(four_var_models[key], typ=3)
    print(results)
    # R^2 = (intercept + explanatory variable(s) SS) / total SS
    # (total-error)/total -> 1 - error/total
    r_squared = 1 - results["sum_sq"]["Residual"]/results["sum_sq"].sum()
    print("R^2 =", round(r_squared,3), "\n")

                   sum_sq    df          F    PR(>F)
Intercept       46.545953   1.0  19.668141  0.000063
region          35.922544   3.0   5.059729  0.004331
single_parent   40.590960   1.0  17.151840  0.000158
metropolitan    30.753143   1.0  12.994839  0.000807
poverty          5.624743   1.0   2.376753  0.130481
Residual       101.762336  43.0        NaN       NaN
R^2 = 0.61 

                   sum_sq    df          F    PR(>F)
Intercept        0.628138   1.0   0.266566  0.608290
region          34.740765   3.0   4.914368  0.005045
single_parent   47.174203   1.0  20.019542  0.000055
metropolitan    20.840934   1.0   8.844367  0.004805
high_school      6.061548   1.0   2.572368  0.116066
Residual       101.325531  43.0        NaN       NaN
R^2 = 0.519 

                   sum_sq    df          F    PR(>F)
Intercept       21.164969   1.0   8.478126  0.005677
region          36.346582   3.0   4.853159  0.005381
single_parent   61.419508   1.0  24.603028  0.000012
metropolitan    21.

In [43]:
print(sm.stats.anova_lm(present_model, four_var_models["poverty"], test="F"))

   df_resid         ssr  df_diff   ss_diff         F    Pr(>F)
0      44.0  107.387079      0.0       NaN       NaN       NaN
1      43.0  101.762336      1.0  5.624743  2.376753  0.130481


Here we have an insignificant $p$-value, so I'll stop with three explanitory variables. 

So here is the model I got:
$$
murder.rate = -8.44 + -2.29 \times Northwest + 0.51 \times South + -0.24 \times West + 0.47 \times single.parent + 0.04 \times metropolitan
$$

Below are the details of the model.

In [ ]:
print(present_model.summary())